In [ ]:
%load_ext autoreload
%autoreload 2
import sys
import os
from decimal import *

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import allantools

sys.path.append('../../qerr')
sys.path.append('qerr')
sys.path.append('quabo/pano_software')
sys.path.append('quabo/pano_software/util')

from dgnss_utils import panoseti_file_interfaces as pfi
from dgnss_utils import pff
from qerr_utils import *

In [ ]:
def load_ph_data(data_dir, run_dir, module_id):
    """
    Create a pd.DataFrame containing the header information of every PH event in data_dir/run_dir for module_id.
    """
    ori = pfi.ObservingRunInterface(data_dir, run_dir)
    if module_id not in ori.obs_pff_files:
        print(f'No module with ID "{module_id}"\n'
              f'Available module_ids:\n\t{list(ori.obs_pff_files.keys())}')
        return
    columns = [
        'wr_timestamp (s)',
        'quabo_num',
        'pkt_num',
        'pkt_tai',
        'pkt_nsec',
        'tv_sec',
        'tv_usec',
        # 'img_data'
    ]
    ph_df = pd.DataFrame(columns=columns)
    
    for ph_file in ori.obs_pff_files[module_id]["ph"]:
        fname = ph_file["fname"]
        fpath = f'{data_dir}/{run_dir}/{fname}'
        with open(fpath, 'rb') as fp:
            frame_iterator = ori.pulse_height_frame_iterator(fp, 1)
            for j, img in frame_iterator:
                j['wr_timestamp (s)'] = pff.wr_to_unix_decimal(j['pkt_tai'], j['pkt_nsec'], j['tv_sec'])
                # j['img_data'] = img
                ph_df.loc[len(ph_df)] = j
    ph_df['unix_timestamp'] = pd.to_datetime(ph_df['wr_timestamp (s)'].astype(np.float64), unit = 's', utc=True)
    ph_df = ph_df.sort_values(by='wr_timestamp (s)')
    return ph_df


def load_qerr_df(fpath):
    """
    Load the pd.DataFrame containing quantization error (qerr) data produced by qerr/capture_qerr.py
    """
    df = load_data(fpath)
    df['unix_timestamp'] = pd.to_datetime(df['pkt_unix_timestamp_TIM-TP'], unit='ns')
    # qerr data timestamps are in pacific time
    df['unix_timestamp'] = df['unix_timestamp'].dt.tz_localize('US/Pacific').dt.tz_convert('utc')
    df['qErr (ns)'] = df['qErr (ps)'] * 1e-3
    df = df.sort_values(by='unix_timestamp')
    return df


def align_data(q0, qerr0, q1, qerr1):
    """
    Aligns PH-event and qerr pd.DataFrames to their earliest common timestamp and latest common timestamp. 
    """
    tsts = {
        'q0': q0,
        'qerr0': qerr0,
        'q1': q1,
        'qerr1': qerr1,
    } 
    ealiest_common_timestamp = max(map(lambda s: s['unix_timestamp'].round('ms').min(), tsts.values()))
    latest_common_timestamp = min(map(lambda s: s['unix_timestamp'].round('ms').max(), tsts.values()))

    def trim(df):
        start_idx = np.searchsorted(df['unix_timestamp'], ealiest_common_timestamp, side='right')
        end_idx = np.searchsorted(df['unix_timestamp'], latest_common_timestamp, side='left')
        return df.iloc[start_idx:end_idx].reset_index(drop=True)
        
    aq0, aqerr0, aq1, aqerr1 = trim(q0), trim(qerr0), trim(q1), trim(qerr1)
    new_len = min(len(aq0), len(aqerr0), len(aq1), len(aqerr1))
    return aq0[:new_len], aqerr0[:new_len], aq1[:new_len], aqerr1[:new_len]

In [ ]:
data_dir = 'quabo/data'

plt_num = 0

if plt_num == 0:
    # WR GrandMaster Mode
    run_dir = 'obs_UCB_lab.start_2024-07-08T20:15:59Z.runtype_eng-test.pffd'
    qerr0_fpath = 'qerr/data/quabo_qerr_data/data/start_2024-07-08T13:15:47.088459.device_ttyUSB0/data-type_MERGED.start_2024-07-08T13:15:47.088459'
    qerr1_fpath = 'qerr/data/quabo_qerr_data/data/start_2024-07-08T13:15:53.105972.device_ttyUSB1/data-type_MERGED.start_2024-07-08T13:15:53.105972'
elif plt_num == 1:
    # WR Master Mode
    run_dir = 'obs_UCB_lab.start_2024-07-08T18:51:45Z.runtype_eng-test.pffd'
    qerr0_fpath = 'qerr/data/quabo_qerr_data/data/start_2024-07-08T11:51:35.102744.device_ttyUSB0/data-type_MERGED.start_2024-07-08T11:51:35.102744'
    qerr1_fpath = 'qerr/data/quabo_qerr_data/data/start_2024-07-08T11:51:39.088319.device_ttyUSB1/data-type_MERGED.start_2024-07-08T11:51:39.088319'


qerr0 = load_qerr_df(qerr0_fpath)
q0 = load_ph_data(data_dir, run_dir, 253)

qerr1 = load_qerr_df(qerr1_fpath)
q1 = load_ph_data(data_dir, run_dir, 254)

pfi.ObservingRunInterface(data_dir, run_dir).obs_pff_files

In [ ]:
# Align data
aq0, aqerr0, aq1, aqerr1 = align_data(q0, qerr0, q1, qerr1)

for df in [aq0, aqerr0, aq1, aqerr1]:
    # print(len(df))
    # print(df['unix_timestamp'].iloc[0].round('ms'))
    print(df['unix_timestamp'].iloc[-1].round('ms'))

# Compute differences

diffs = pd.DataFrame(columns=['quabo_wr_diffs', 'qerr_diffs', 'quabo_wr_diffs_without_qerrs'])

diffs['quabo_wr_diffs (ns)'] = ((aq0['wr_timestamp (s)'] - aq1['wr_timestamp (s)']) * 10**9).astype(np.float64)  # Do second -> nanosecond conversion with Decimal to elimiate floating-point errors
diffs['qerr_diffs (ns)'] = (aqerr1['qErr (ns)'] - aqerr0['qErr (ns)'])
diffs['quabo_wr_diffs_without_qerrs (ns)'] = diffs['quabo_wr_diffs (ns)'] - diffs['qerr_diffs (ns)']

print()
print(diffs.describe())

In [ ]:
# Create plot

start_timestamp = aq0['unix_timestamp'][0].isoformat()

total_time = len(aq0)
time_per_window = total_time
nsubplots = total_time // time_per_window

plot_qerr = False

xtick_spacing = 20
ax_height = 5
ax_width = 20

f, axs = plt.subplots(nsubplots, 1, figsize=(ax_width, ax_height * nsubplots))

for i in range(nsubplots):
    t_low = time_per_window * i
    t_high = time_per_window * (i + 1)
    diffs_in_window = diffs[t_low: t_high]
    
    if nsubplots > 1:
        ax = axs[i]
    else:
        ax = axs
    
    sns.lineplot(
        diffs_in_window['quabo_wr_diffs (ns)'],
        label='1PPS Delta', 
        ax=ax
    )
    sns.lineplot(
        diffs_in_window['quabo_wr_diffs_without_qerrs (ns)'],
        label='(1PPS Delta) - (qErr Delta)',
        ax=ax,
        linewidth = 2
    )
    if plot_qerr:
        sns.lineplot(
            diffs_in_window['qerr_diffs (ns)'],
            label='qErr Delta',
            ax=ax
        )

    # Style axis
    
    ax.set_xlabel('Elapsed Time (s)')
    if xtick_spacing and time_per_window != total_time:
        ax.set_xticks(np.arange(t_low, t_high + xtick_spacing, xtick_spacing))
    # ax.set_xlim([t_low, t_high])
    
    ax.set_ylabel('1PPS Delta (ns)')
    # ax.set_yticks(np.arange(-7.5, 27.5, 2.5))
    
    ax.grid(True)
    
    if i == 0:
        ax.set_title(f'1PPS Delta (ns) vs Elapsed Time (s); start = {start_timestamp}')
    # else:
    #     ax.get_legend().remove();


plt.figure(figsize=(12,6))
# plt.grid(True)
sns.histplot(diffs['quabo_wr_diffs (ns)'], stat='density', label='1PPS Delta', common_bins=False);
sns.histplot(diffs['quabo_wr_diffs_without_qerrs (ns)'], stat='density', label='(1PPS) - (qErr Delta)');

plt.legend()
# plt.axvline(x=np.mean(data), label='mean', c='r')
# plt.axvline(x=np.median(data), label='median', c='b')
plt.title('Distribution of Errors')
plt.xlabel('Time Difference (ns)');

In [ ]:
# Code for testing TIE
def get_clkerr(quabo_data):
    ideal_clock_period = 1
    clkerr = pd.DataFrame(columns = ['ideal clk time', 'actual clk time', 'actual clk period', 'period change', 'time interval error'])
    clkerr['ideal clk time'] = np.arange(len(quabo_data))
    clkerr['actual clk time'] = (quabo_data['wr_timestamp (s)'] - quabo_data['wr_timestamp (s)'].min()).astype(np.float64)
    clkerr['time interval error (s)'] = clkerr['actual clk time'] - clkerr['ideal clk time']
    clkerr['actual clk period (s)'] = clkerr['actual clk time'].diff()
    clkerr['period change'] = clkerr['actual clk period'].diff()
    return clkerr

clkerr0 = get_clkerr(aq0)
clkerr1 = get_clkerr(aq1)
print(clkerr0['actual clk period (s)'].describe(), '\n' * 2, clkerr1['actual clk period (s)'].describe())

In [ ]:
f, axs = plt.subplots(2, 2, figsize=(20, 10))
f.suptitle(f'Quabo timing stability (start = {start_timestamp}')

low = 0
high = len(clkerr0)

sns.lineplot(clkerr0['actual clk period (s)'][low:high], ax=axs[0, 0], label='Q0')
sns.lineplot(clkerr1['actual clk period (s)'][low:high], ax=axs[0, 0], label='Q1')
sns.histplot(clkerr0['actual clk period (s)'][low:high], ax=axs[0, 1], stat='density', label='Q0')
sns.histplot(clkerr1['actual clk period (s)'][low:high], ax=axs[0, 1], stat='density', label='Q1')

sns.lineplot(clkerr0['time interval error (s)'][low:high], ax=axs[1, 0], label='Q0')
sns.lineplot(clkerr1['time interval error (s)'][low:high], ax=axs[1, 0], label='Q1')
sns.histplot(clkerr0['time interval error (s)'][low:high], ax=axs[1, 1], stat='density', label='Q0')
sns.histplot(clkerr1['time interval error (s)'][low:high], ax=axs[1, 1], stat='density', label='Q1')


axs[0, 0].grid(True)
axs[1, 0].grid(True)


axs[0, 0].set_title('Clock period per 1PPS sample')
axs[0, 1].set_title('Distribution of clock periods')

axs[1, 0].set_title('Time interval error (s) per 1PPS sample')
axs[1, 1].set_title('Distribution of Time interval errors (s)');

In [ ]:
tdev_taus0, tdev0, tdev_error0, tdev_ns0 = allantools.tdev(clkerr0['actual clk time'], taus = "all")
mtie_taus0, mtie0, mtie_error0, mtie_ns0 = allantools.mtie(clkerr0['actual clk time'], taus = "all")

tdev_taus1, tdev1, tdev_error1, tdev_ns1 = allantools.tdev(clkerr1['actual clk time'], taus = "all")
mtie_taus1, mtie1, mtie_error1, mtie_ns1 = allantools.mtie(clkerr1['actual clk time'], taus = "all")

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(15, 5), dpi=500)
f.suptitle(f'Frequency Stability (Start time: {start_timestamp})')

sns.lineplot(x=tdev_taus0, y=tdev0, ax=axs[0], label='Q0')
sns.lineplot(x=tdev_taus1, y=tdev1, ax=axs[0], label='Q1')

sns.lineplot(x=mtie_taus0, y=mtie0, ax=axs[1], label='Q0')
sns.lineplot(x=mtie_taus1, y=mtie1, ax=axs[1], label='Q1')

plt.rcParams['text.usetex'] = True
for ax in axs:
    ax.set_xlabel('Averaging Time (s)')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.grid(True, which='both')

axs[0].set_title('TDEV')
axs[0].set_ylabel(f'Modified Allen Variance, $\\log($ Mod $\\sigma_y(\\tau))$')

axs[1].set_title('MTIE')
axs[1].set_ylabel('Maximum Time Interval Error (s)');

In [ ]:
a = allantools.Dataset(data=clkerr0['actual clk time'],data_type="phase", taus="all")

a.compute('tdev')
b = allantools.Plot()
b.plot(a, errorbars=True, grid=True)
# b.ax.axis([1e-3, 10, 1e-6, 1])    
b.ax.set_xlabel("Tau (s)")
b.show()